In [1]:
import numpy as np
import pandas as pd
import joblib 
import os
import matplotlib.pyplot as plt
import warnings;warnings.simplefilter('ignore')
from numpy.linalg import lstsq,inv

In [2]:
root = '../data/phase_2/cleaned'
os.listdir(root)

['col_names.pkl', 'train(energy).csv', 'train.csv']

In [3]:
col_names = joblib.load(root+'/col_names.pkl')

In [4]:
df = pd.read_csv(root+'/train.csv',index_col=0)

In [5]:
df[col_names['case']].head()

,Case Conditions_Feed Rate to C013 (Input)_m3/hr,Case Conditions_Heart Cut Prod. Rate (Input)_m3/hr,Case Conditions_C6P- in Heart Cut (Input)_wt%,Case Conditions_C7+ in Light End (Input)_wt%
001-01,10.0,100.0,1.0,2.5
001-02,10.0,100.0,1.0,5.0
001-03,10.0,100.0,1.0,10.0
001-04,10.0,100.0,1.5,2.5
001-05,10.0,100.0,1.5,5.0


# select df

In [6]:
df = df[(df[col_names['case']].values==[10,100,2.5,1.0]).all(axis=1)].sample(54)
print(df.shape)
df.head()

ValueError: a must be greater than 0 unless no samples are taken

# get xna fna xle fle

In [7]:
xna = df[col_names['xna']].values
fna = df[col_names['fna']].values
#================================
xle = df[col_names['xle']].values
fle = df[col_names['fle']].values

# Xna@W = Xle

# W = inv(Xna)@Xle ?

In [8]:
W = np.matmul(inv(xna),xle)
W

LinAlgError: Last 2 dimensions of the array must be square

# lstsq

In [9]:
n_col = [ i.split('_')[-2] for i in col_names['xna']]
def lstsq_sol(A,B):
    x = np.array([np.concatenate((v,[1])) for v in A])
    y = B
    w,error,_,_ = np.linalg.lstsq(x,y)
    y_pred = np.dot(x,w)
    #for i,name in enumerate(n_col):
    #    plt.plot(y[:,i],label='real_{}'.format(name))
    #    plt.plot(y_pred[:,i],label='pred{}'.format(name))
    #    plt.legend()
    #    plt.show()
    res = pd.DataFrame(index=n_col,columns=['MSE'])
    for i,name in enumerate(n_col):
        res.loc[name,'MSE'] = np.sum((y_pred[:,i]-y[:,i])**2)/len(y_pred)
    res.loc['AVG'] = res.mean(axis=0)
    return res

# sle

In [10]:
xle = df[col_names['xle']].values
fle = df[col_names['fle']].values
A = xna
B = xle
res = lstsq_sol(A,B)
res

,MSE
Oxygen,0
Methane,0
Ethane,0
Propane,0.00905029
n-Butane,0.0337736
i-Pentane,2.10219
n-Pentane,7.08568
tr2-Pentene,0.0013395
Cyclopentane,0.0877308
3-Methylpentane,5.08125


# shc

In [11]:
xhc = df[col_names['xhc']].values
fhc = df[col_names['fhc']].values
A = xna
B = xhc
res = lstsq_sol(A,B)
res

,MSE
Oxygen,0
Methane,0
Ethane,0
Propane,4.73218e-08
n-Butane,9.61468e-07
i-Pentane,0.000234705
n-Pentane,0.00112939
tr2-Pentene,1.9723e-07
Cyclopentane,0.000100299
3-Methylpentane,0.0158417


# she

In [12]:
xhe = df[col_names['xhe']].values
fhe = df[col_names['fhe']].values
A = xna
B = xhe
res = lstsq_sol(A,B)
res

,MSE
Oxygen,0
Methane,0
Ethane,0
Propane,5.08804e-57
n-Butane,5.70071e-43
i-Pentane,1.77949e-31
n-Pentane,9.8502e-29
tr2-Pentene,2.19775e-32
Cyclopentane,5.21987e-25
3-Methylpentane,5.27758e-22
